# [DL-URFU-2024] Построение классифика...

## Dataset Description

Fashion-MNIST – аналог знаменитого датасета MNIST, который состоит не из рукописных символов, а из предметов одежды. Он был создан, чтобы актуализировать учебные задачи в компьютерном зрении и приблизить их к реальности.

Решение более сложной постановки этой задачи может использоваться, например, в маркетплейсах. Покупателю могут рекомендоваться товары по изображениям в соответствии с введённым описанием. Продавцу при загрузке изображений в карточку товара могут автоматически рекомендоваться теги и описание предмета одежды.

Каждое изображение представляет собой набор их 784 пикселей (28x28). Каждый пиксель принимает значение от 0 до 255, обозначая его цвет в черно-белой шкале. Обучающая выборка содержит 786 столбцов, где первый (label) – принадлежность изображения к классу, а последний – его уникальный номер (id). Остальные столбцы – пиксели изображения.
Проведите первичный анализ данных, чтобы определить наличие NaN значений.
Тестовая выборка (на основе которой должно быть получено исследование) содержит все те же столбцы, кроме label.

### Расшифровка классов (label), к которым принадлежат изображения:

0. T-shirt/top – футболка
1. Trouser – брюки
2. Pullover – свитер
3. Dress – платье
4. Coat – пальто
5. Sandal – сандалия
6. Shirt – рубашка
7. Sneaker – кроссовок
8. Bag – сумка
9. Ankle boot – сапог

# Добавляем необходимые библиотеки

In [34]:
!pip install kaggle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
%matplotlib inline

# Загружаем DataSet с Kaggle соревнование DL-URFU-2024

In [36]:
df_train = pd.read_csv('/content/fashion-mnist_train.csv')
df_test = pd.read_csv('/content/fashion-mnist_test.csv')
submission = pd.read_csv('/content/sample_submission.csv')

Получим размерность обоих датасетов

In [37]:
print(df_train.shape)

(60000, 785)


In [38]:
print(df_test.shape)

(10000, 785)


In [40]:
# Извлечение пиксельных данных
x_train = df_train.drop(['label'], axis=1).values
x_test = df_test.drop(['label'], axis=1).values

# Получение меток
y_train = df_train['label'].values
y_test = df_test['label'].values


In [41]:
classes = ['футболка', 'брюки', 'свитер', 'платье', 'пальто', 'туфли', 'рубашка', 'кроссовки', 'сумка', 'ботинки']

**Преобразование размерности данных в наборе**

In [44]:
x_train = x_train.reshape(60000, 784)

**Нормализация данных**

In [45]:
x_train = x_train / 255

In [46]:
print(y_train[0])

2


**Преобразуем метки в формат one hot encoding**

In [47]:
y_train = utils.to_categorical(y_train, 10)

In [48]:
print(y_train[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


## Создаем нейронную сеть

In [49]:
# Создаем последовательную модель
model = Sequential()
# Входной полносвязный слой, 800 нейронов, 784 входа в каждый нейрон
model.add(Dense(800, input_dim=784, activation="relu"))
# Выходной полносвязный слой, 10 нейронов (по количеству рукописных цифр)
model.add(Dense(10, activation="softmax"))

In [53]:
# Обучаем модель
model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

model.fit(x_train, y_train,
          batch_size=200,
          epochs=100,
          verbose=1,
          validation_split=0.2)

# Сохраняем модель.
model.save("cnn_mnist_classification")

Epoch 1/100
240/240 [==============================] - 4s 17ms/step - loss: 0.3180 - accuracy: 0.8895 - val_loss: 0.3259 - val_accuracy: 0.8875
Epoch 2/100
240/240 [==============================] - 4s 15ms/step - loss: 0.3171 - accuracy: 0.8895 - val_loss: 0.3262 - val_accuracy: 0.8875
Epoch 3/100
240/240 [==============================] - 5s 23ms/step - loss: 0.3162 - accuracy: 0.8902 - val_loss: 0.3275 - val_accuracy: 0.8856
Epoch 4/100
240/240 [==============================] - 4s 15ms/step - loss: 0.3153 - accuracy: 0.8907 - val_loss: 0.3303 - val_accuracy: 0.8847
Epoch 5/100
240/240 [==============================] - 4s 15ms/step - loss: 0.3147 - accuracy: 0.8913 - val_loss: 0.3286 - val_accuracy: 0.8862
Epoch 6/100
240/240 [==============================] - 4s 18ms/step - loss: 0.3139 - accuracy: 0.8918 - val_loss: 0.3257 - val_accuracy: 0.8870
Epoch 7/100
240/240 [==============================] - 5s 20ms/step - loss: 0.3128 - accuracy: 0.8917 - val_loss: 0.3259 - val_accuracy:

**Запускаем распознавание набора данных, на котором обучалась сеть**

In [63]:
predictions = model.predict(x_test)
predicted_classes = predictions.argmax(axis=1)

313/313 [==============================] - 1s 4ms/step


In [65]:
# Сохранение предсказанных меток в колонке 'Category'
submission['Category'] = predicted_classes

# Сохранение результатов в CSV
submission.to_csv('/content/result_submission.csv', index=False)